<a href="https://colab.research.google.com/github/ankit-quant/Data-Harrvesting--1/blob/main/Webinar_Data_Harvesting_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Agenda - Part - 1
1. Understand KiteConnect Login
2. Automate KiteConnect Login
3. Understand Jugaad Trader & Login

In [1]:
import pandas as pd
import numpy as np
import os
import pyotp
import datetime as dt
import json
import time
import requests

from kiteconnect import KiteConnect

In [2]:
# UserName
# Password
# TOTP Seed/Key
creds = {'user_id':'ET5487','password':'StoicEmperor@!77','totp_key':'JXLCTUCANDYLJN7FLRLVU6DSFXQKQ6YV','api_key':'elrfps73mpn9aou4', 'api_secret':'tqoel0f8kg0yzkipd5d8wee0b3our5yr' }

In [3]:
base_url = "https://kite.zerodha.com"
login_url = "https://kite.zerodha.com/api/login"
twofa_url = "https://kite.zerodha.com/api/twofa"
instruments_url = "https://api.kite.trade/instruments"

##### Login using Zerodha Credentials

In [19]:
session = requests.Session()
response = session.post(login_url,data={'user_id':creds['user_id'],'password':creds['password']})
request_id = json.loads(response.text)['data']['request_id']
twofa_pin = pyotp.TOTP(creds['totp_key']).now()
response_1 = session.post(twofa_url,data={'user_id':creds['user_id'],'request_id':request_id,'twofa_value':twofa_pin,'twofa_type':'totp'})
kite = KiteConnect(api_key=creds['api_key'])
kite_url = kite.login_url()

In [20]:
try:
  session.get(kite_url)
except Exception as e:
  e_msg = str(e)
  #print(e_msg)
  request_token = e_msg.split('request_token=')[1].split(' ')[0].split('&action')[0]
  print('Successful Login with Request Token:{}'.format(request_token))

access_token = kite.generate_session(request_token,creds['api_secret'])['access_token']
kite.set_access_token(access_token)

Successful Login with Request Token:AArBK6Ac1ZNKdSP00KGDcsECDpXs3a4v


In [21]:
access_token

'V5R3rPTLAnjoBcWFnSfcJTu8lKtkmB7j'

In [23]:
pd.DataFrame(kite.orders())

,placed_by,order_id,exchange_order_id,parent_order_id,status,status_message,status_message_raw,order_timestamp,exchange_update_timestamp,exchange_timestamp,...,price,trigger_price,average_price,filled_quantity,pending_quantity,cancelled_quantity,market_protection,meta,tag,guid
0,ET5487,230105004219939,1100000024139742,None,COMPLETE,None,None,2023-01-05 14:19:11,2023-01-05 14:19:11,2023-01-05 14:19:11,...,0,0,332.25,1,0,0,0,{},None,01X2GWYpc9K3vLV
1,ADMINSQF,230105005204292,1100000028497423,None,COMPLETE,None,None,2023-01-05 15:21:05,2023-01-05 15:21:05,2023-01-05 15:21:05,...,0,0,333.50,1,0,0,0,{},None,None


#### Replicating Zerodha Login using Jugaad Trader

In [35]:
from jugaad_trader import Zerodha

In [48]:
kite1 = Zerodha()
kite1.user_id=creds['user_id']
kite1.password = creds['password']
json_res = kite1.login_step1()
twofa = pyotp.TOTP(creds['totp_key']).now()
kite1.twofa=twofa
json_res_1 = kite1.login_step2(json_res)
kite1.enc_token = kite1.r.cookies['enctoken']

In [51]:
pd.DataFrame(kite1.orders())

,placed_by,order_id,exchange_order_id,parent_order_id,status,status_message,status_message_raw,order_timestamp,exchange_update_timestamp,exchange_timestamp,...,price,trigger_price,average_price,filled_quantity,pending_quantity,cancelled_quantity,market_protection,meta,tag,guid
0,ET5487,230105004219939,1100000024139742,None,COMPLETE,None,None,2023-01-05 14:19:11,2023-01-05 14:19:11,2023-01-05 14:19:11,...,0,0,332.25,1,0,0,0,{},None,01X2GWYpc9K3vLV
1,ADMINSQF,230105005204292,1100000028497423,None,COMPLETE,None,None,2023-01-05 15:21:05,2023-01-05 15:21:05,2023-01-05 15:21:05,...,0,0,333.50,1,0,0,0,{},None,None
